In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

# Crear carpeta de salida
OUTPUT_DIR = "modelo_rf_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Cargar archivos CSV
csv_dir = "../dataset_landmarks"
csv_files = glob(os.path.join(csv_dir, "*.csv"))
np.random.seed(42)
np.random.shuffle(csv_files)

split_idx = int(0.8 * len(csv_files))
train_files = csv_files[:split_idx]
test_files = csv_files[split_idx:]

train_df = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True)
test_df = pd.concat([pd.read_csv(f) for f in test_files], ignore_index=True)

# 2. Preprocesamiento
# Guardar nombres de columnas originales para análisis posterior
original_columns = train_df.columns.tolist()
with open(os.path.join(OUTPUT_DIR, "column_names.json"), 'w') as f:
    json.dump(original_columns, f)

# Eliminar columnas de visibilidad y frame
for df_ in [train_df, test_df]:
    df_.drop(columns=[c for c in df_.columns if 'vis' in c or c == 'frame'], inplace=True)

# Codificar etiquetas
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
test_df['label'] = le.transform(test_df['label'])

# Guardar mapeo de etiquetas para referencia 
label_mapping = {str(cls): int(idx) for cls, idx in zip(le.classes_, le.transform(le.classes_))}
with open(os.path.join(OUTPUT_DIR, "label_mapping.json"), 'w') as f:
    json.dump(label_mapping, f)

# Separar características y etiquetas
X_train, y_train = train_df.drop(columns=['label']), train_df['label']
X_test, y_test = test_df.drop(columns=['label']), test_df['label']

# 3. Normalización de landmarks respecto a la cadera central (landmark 0)
def normalize_landmarks(df):
    """
    Normaliza todas las coordenadas x,y respecto al landmark 0 (cadera central)
    """
    normalized_df = df.copy()
    
    # Identificar columnas de coordenadas
    x_cols = [col for col in df.columns if '_x' in col]
    y_cols = [col for col in df.columns if '_y' in col]
    
    # Usar el primer landmark (0) como referencia
    reference_x_col = '0_x'
    reference_y_col = '0_y'
    
    if reference_x_col in df.columns and reference_y_col in df.columns:
        for idx, row in df.iterrows():
            # Obtener valores de referencia
            ref_x = row[reference_x_col]
            ref_y = row[reference_y_col]
            
            # Normalizar coordenadas X
            for x_col in x_cols:
                normalized_df.loc[idx, x_col] = row[x_col] - ref_x
                
            # Normalizar coordenadas Y
            for y_col in y_cols:
                normalized_df.loc[idx, y_col] = row[y_col] - ref_y
    
    return normalized_df

# Aplicar normalización
print("Aplicando normalización respecto a la cadera central...")
X_train_normalized = normalize_landmarks(X_train)
X_test_normalized = normalize_landmarks(X_test)

# 4. Aumentación de datos (crear variaciones con pequeño ruido para mejorar robustez)
def augment_data(X, y, noise_level=0.005, copies=1):
    """
    Añade variaciones con pequeño ruido para mejorar la robustez del modelo
    """
    X_aug_list = [X]
    y_aug_list = [y]
    
    # Crear copias con ruido para cada clase
    for _ in range(copies):
        for class_id in np.unique(y):
            # Seleccionar muestras de esta clase
            class_mask = y == class_id
            X_class = X.loc[class_mask].copy()
            y_class = y.loc[class_mask].copy()
            
            # Generar ruido
            noise = np.random.normal(0, noise_level, X_class.shape)
            
            # Aplicar ruido solo a las coordenadas x,y (no a z)
            for col in X_class.columns:
                if '_x' in col or '_y' in col:
                    X_class[col] += pd.Series(noise[:, X_class.columns.get_loc(col)])
            
            # Añadir a los datos aumentados
            X_aug_list.append(X_class)
            y_aug_list.append(y_class)
    
    # Concatenar todos los datos
    X_augmented = pd.concat(X_aug_list, ignore_index=True)
    y_augmented = pd.concat(y_aug_list, ignore_index=True)
    
    return X_augmented, y_augmented

# Aplicar aumentación de datos
print("Aplicando aumentación de datos...")
X_train_aug, y_train_aug = augment_data(X_train_normalized, y_train, copies=1)
print(f"Tamaño original: {len(X_train)} muestras")
print(f"Tamaño después de aumentación: {len(X_train_aug)} muestras")

# 5. Escalado de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_aug)
X_test_scaled = scaler.transform(X_test_normalized)

# 6. Entrenamiento con búsqueda de hiperparámetros (Grid Search)
print("\nEntrenando modelo con búsqueda de hiperparámetros...")
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_leaf': [2, 5, 10],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)
grid = GridSearchCV(rf, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1)
grid.fit(X_train_scaled, y_train_aug)

best_rf = grid.best_estimator_
print("\nMejores hiperparámetros encontrados:")
print(grid.best_params_)

# 7. Evaluación del modelo
print("\nEvaluando modelo en conjunto de prueba...")
y_pred = best_rf.predict(X_test_scaled)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"\nMétricas de rendimiento:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# 8. Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("Matriz de Confusión - Random Forest")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "confusion_matrix.png"))
plt.close()

# 9. Análisis de importancia de características
importances = best_rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Convertir X_train_aug a DataFrame si es un array
if not isinstance(X_train_aug, pd.DataFrame):
    X_train_aug = pd.DataFrame(X_train_aug, columns=X_train.columns)

# Top 15 características
plt.figure(figsize=(12, 6))
top_n = min(15, len(importances))
plt.title(f'Top {top_n} características más importantes')
plt.bar(range(top_n), importances[indices[:top_n]], align='center')
plt.xticks(range(top_n), X_train_aug.columns[indices[:top_n]], rotation=90)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "top_features.png"))
plt.close()

# Todas las características 
if len(importances) > 30:
    # Dividir en múltiples gráficos si hay muchas características
    chunk_size = 30
    for i in range(0, len(importances), chunk_size):
        end = min(i + chunk_size, len(importances))
        plt.figure(figsize=(15, 8))
        plt.title(f'Importancia de características (grupo {i//chunk_size + 1})')
        plt.bar(range(end-i), importances[indices[i:end]], align='center')
        plt.xticks(range(end-i), X_train_aug.columns[indices[i:end]], rotation=90)
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, f"feature_importance_group_{i//chunk_size + 1}.png"))
        plt.close()
else:
    # Un solo gráfico si hay pocas características
    plt.figure(figsize=(15, 8))
    plt.title('Importancia de todas las características')
    plt.bar(range(len(importances)), importances[indices], align='center')
    plt.xticks(range(len(importances)), X_train_aug.columns[indices], rotation=90)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "all_features_importance.png"))
    plt.close()

# Análisis de ángulos y docs
def calculate_joint_angles(df):
    """
    Calcula y analiza ángulos entre articulaciones clave.
    Esto es útil para documentación y análisis.
    """
    angles_info = {}
    
    # Definir landmarks para ángulos de interés (ajustar según MediaPipe Pose)
    # Ejemplo: Rodilla derecha (cadera-rodilla-tobillo)
    # Nota: Estos índices deben ajustarse según la estructura de MediaPipe
    # https://developers.google.com/mediapipe/solutions/vision/pose_landmarker
    joint_angles = {
        "rodilla_derecha": {
            "landmarks": [24, 26, 28],  # Cadera derecha, rodilla derecha, tobillo derecho
            "description": "Ángulo de la rodilla derecha (cadera-rodilla-tobillo)"
        },
        "rodilla_izquierda": {
            "landmarks": [23, 25, 27],  # Cadera izquierda, rodilla izquierda, tobillo izquierdo
            "description": "Ángulo de la rodilla izquierda (cadera-rodilla-tobillo)"
        },
        "codo_derecho": {
            "landmarks": [12, 14, 16],  # Hombro derecho, codo derecho, muñeca derecha
            "description": "Ángulo del codo derecho (hombro-codo-muñeca)"
        },
        "codo_izquierdo": {
            "landmarks": [11, 13, 15],  # Hombro izquierdo, codo izquierdo, muñeca izquierda
            "description": "Ángulo del codo izquierdo (hombro-codo-muñeca)"
        },
        "torso": {
            "landmarks": [11, 23, 24],  # Hombro izquierdo, cadera izquierda, cadera derecha
            "description": "Inclinación del torso"
        }
    }
    
    # Documentar los ángulos analizados para referencia
    with open(os.path.join(OUTPUT_DIR, "joint_angles_reference.json"), 'w') as f:
        json.dump(joint_angles, f, indent=4)
    
    return angles_info

# Documentar información de ángulos para referencia
calculate_joint_angles(X_train)

# Guardar modelo y transformadores
print("\nGuardando modelo y transformadores...")
joblib.dump(best_rf, os.path.join(OUTPUT_DIR, "modelo_rf.pkl"))
joblib.dump(scaler, os.path.join(OUTPUT_DIR, "scaler.pkl"))
joblib.dump(le, os.path.join(OUTPUT_DIR, "label_encoder.pkl"))

# Guardar métricas de evaluación
metrics = {
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'f1': float(f1),
    'best_params': grid.best_params_
}

with open(os.path.join(OUTPUT_DIR, "metrics.json"), 'w') as f:
    json.dump(metrics, f, indent=4)

print(f"\nTodo guardado en la carpeta '{OUTPUT_DIR}'")
print("\nEntrenamiento completado con éxito!")

Aplicando normalización respecto a la cadera central...
Aplicando aumentación de datos...
Tamaño original: 12324 muestras
Tamaño después de aumentación: 24648 muestras

Entrenando modelo con búsqueda de hiperparámetros...
Fitting 5 folds for each of 54 candidates, totalling 270 fits


KeyboardInterrupt: 